## 1. Current weather

### 1.1 obtain current weather(without llm prompt)

In [14]:
!pip install pyowm

import warnings
warnings.filterwarnings("ignore")

In [15]:
import os
from getpass import getpass

from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain_community.utilities import OpenWeatherMapAPIWrapper

#environment variables
os.environ['OPENWEATHERMAP_API_KEY'] = getpass('Type your openweatherapi key: ')
os.environ['OPENAI_API_KEY'] = getpass('OPENAI key here: ')

#show current weather(not used with llm)
weather = OpenWeatherMapAPIWrapper()

ValidationError: 1 validation error for OpenWeatherMapAPIWrapper
  Value error, Did not find openweathermap_api_key, please add an environment variable `OPENWEATHERMAP_API_KEY` which contains it, or pass `openweathermap_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

### 1.2 Get the llm response without the weather agent --> gives the wrong answer

this is due to the fact, that given only the parametric knowledge, llm can't tell today's weather

In [37]:
from langchain.prompts import PromptTemplate

template ='''Question : {question}
Answer: Give short answer'''

prompt = PromptTemplate(template=template,input_variables=["question"])
llm_0 = ChatOpenAI(temperature=0.0)
llm_chain = prompt | llm_0
question = 'What is the weather like right now in Belgrade , Serbia?'

print(llm_chain.invoke(question).content)

The weather in Belgrade, Serbia right now is partly cloudy with a temperature of 22°C.


### 1.3 Use with the llm --> gives correct answer

In [34]:
from langchain.agents import initialize_agent

tools = load_tools(['openweathermap-api'])
tools

#instantiate model
llm = ChatOpenAI(temperature=0.0)

#instantiate agent
weather_agent = initialize_agent(
    tools = tools,
    llm=llm,
    verbose = True,
    max_iterations = 3
)

weather_agent('what is the weather now in Belgrade and in Moscow?')

C:\Users\Nosac\AppData\Local\Temp\ipykernel_18836\1860403288.py:10: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  weather_agent = initialize_agent(
C:\Users\Nosac\AppData\Local\Temp\ipykernel_18836\1860403288.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  weather_agent('what is the weather now in Belgrade and in Moscow?')




> Entering new AgentExecutor chain...
We need to use the open_weather_map tool to fetch the current weather information for both Belgrade and Moscow.
Action: open_weather_map
Action Input: Belgrade
Observation: In Belgrade, the current weather is as follows:
Detailed status: clear sky
Wind speed: 2.57 m/s, direction: 310°
Humidity: 65%
Temperature: 
  - Current: 12.89°C
  - High: 12.96°C
  - Low: 11.35°C
  - Feels like: 11.93°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:We need to get the weather information for Moscow now.
Action: open_weather_map
Action Input: Moscow
Observation: In Moscow, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 5.96 m/s, direction: 37°
Humidity: 58%
Temperature: 
  - Current: 10.79°C
  - High: 11.1°C
  - Low: 10.29°C
  - Feels like: 9.44°C
Rain: {}
Heat index: None
Cloud cover: 100%
Thought:I now know the final answer
Final Answer: The current weather in Belgrade is clear sky with a temperature of 12.89°C and in Mosc

{'input': 'what is the weather now in Belgrade and in Moscow?',
 'output': 'The current weather in Belgrade is clear sky with a temperature of 12.89°C and in Moscow, it is overcast clouds with a temperature of 10.79°C.'}

### 1.5 add llm as another tool

in case the user wants to get the knowledge irrelevant to the tool

In [36]:
from langchain.agents import Tool

# try out non-weather related question 
weather_agent("What is the vangogh's place of birth?")

# let's add the llm itself as a tool to obtain general answer
llm_tool = Tool(
    name='language_model',
    func=llm.predict,
    description='Answers the general question'
)

#add to tools
if llm_tool not in tools:
    tools.append(llm_tool)

weather_agent_v2 = initialize_agent(
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

weather_agent_v2("what is van gogh's place of birth and death?")




> Entering new AgentExecutor chain...
I need to find the location of Vincent van Gogh's birthplace.
Action: open_weather_map
Action Input: Zundert,NL
Observation: In Zundert,NL, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 9.22 m/s, direction: 208°
Humidity: 90%
Temperature: 
  - Current: 14.96°C
  - High: 15.66°C
  - Low: 14.38°C
  - Feels like: 14.86°C
Rain: {}
Heat index: None
Cloud cover: 100%
Thought:The weather information for Zundert, NL has been retrieved, but I need to find the specific location of Vincent van Gogh's birthplace.
Action: open_weather_map
Action Input: Zundert,NL
Observation: In Zundert,NL, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 9.22 m/s, direction: 208°
Humidity: 90%
Temperature: 
  - Current: 14.96°C
  - High: 15.66°C
  - Low: 14.38°C
  - Feels like: 14.86°C
Rain: {}
Heat index: None
Cloud cover: 100%
Thought:I need to search for the specific location of Vincent van Gogh's birthplace

{'input': "what is van gogh's place of birth and death?",
 'output': 'Vincent van Gogh was born in Zundert, Netherlands and died in Auvers-sur-Oise, France.'}

### 1.6 Can't provide the data not accesible via tool api

In [38]:
weather_agent('Can you retrieve the historical data about the temperatures for Belgrade, Serbia during the last month daily? ')



> Entering new AgentExecutor chain...
I need to use the open_weather_map tool to fetch historical temperature data for Belgrade, Serbia.
Action: open_weather_map
Action Input: Belgrade,RS
Observation: In Belgrade,RS, the current weather is as follows:
Detailed status: clear sky
Wind speed: 2.57 m/s, direction: 300°
Humidity: 67%
Temperature: 
  - Current: 12.49°C
  - High: 12.81°C
  - Low: 10.79°C
  - Feels like: 11.54°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:The current weather information was retrieved successfully, but historical data was not provided. I need to find a different tool to retrieve historical temperature data.
Final Answer: No, I cannot retrieve historical temperature data for Belgrade, Serbia using the open_weather_map tool.

> Finished chain.


{'input': 'Can you retrieve the historical data about the temperatures for Belgrade, Serbia during the last month daily? ',
 'output': 'No, I cannot retrieve historical temperature data for Belgrade, Serbia using the open_weather_map tool.'}

## 2. WebPage scraper
let's retrieve the important information from the website. We'll use the smash burgers beograd site https://wolt.com/en/srb/belgrade/restaurant/smash-burgerss

In [56]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://wolt.com/en/srb/belgrade/restaurant/smash-burgerss')

data = loader.load()
data

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://wolt.com/en/srb/belgrade/restaurant/smash-burgerss', 'title': 'Smash Burgers | Wolt | Delivery | Belgrade', 'description': 'Order now online from the menu | The first SMASH burgers in Belgrade!', 'language': 'en'}, page_content='\n\n\n\n\n\nSmash Burgers | Wolt | Delivery | Belgrade\n\n \n\n\n\n\n\n\n\n\n\n\n  \n\n\n0View orderRSD\xa00.00Smash BurgersThe first SMASH burgers in Belgrade!Order togetherMEGA POPUSTISPECIAL OFFER - WESTERN SMASH BURGERSBLUE CHEESE & BACON SMASH BURGERSCLASSIC SMASH  BURGERSORIGINAL SMASH BURGERSBBQ JALAPENO SMASH BURGERSPRILOZI & SOSOVIBROWNIEPIĆEThis menu is in Serbian. Would you like to view a machine translation in another language?TranslateMEGA POPUSTITriple Original Smash burger3 x 60g junećeg mesa, cheddar, iceberg, luk, krastavac, burger sosRSD\xa0870.00SPECIAL OFFER - WESTERN SMASH BURGERSDouble Western Smash burger2 x 60g junećeg mesa, ementaler, iceberg, luk, hrskava slanina, western sosRSD\xa0760.00Triple We

In [61]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB 330.3 kB/s eta 0:00:45
   ---------------------------------------- 0.0/14.6 MB 330.3 kB/s eta 0:00:45
   ---------------------------------------- 0.1/14.6 MB 654.9 kB/s eta 0:00:23
   ---------------------------------------- 0.2/14.6 MB 751.6 kB/s eta 0:00:20
    --------------------------------------- 0.3/14.6 MB 1.1 MB/s eta 0:00:13
   - -------------------------------------- 0.6/14.6 MB 1.8 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.6 MB 1.8 MB/s eta 0:00:08
   -- ------------------------------------- 0.9/14.6 MB 2.2 MB/s eta 0:00:07
   -- ------------------------------------- 0.9/14.6 MB 2.2 MB/s eta 0:00:07
   -- ------------------------------------- 0.9/14.6 MB 1.8 MB/s eta 0:00:08
   --- -----

In [62]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

#split text into 1000 chunks
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts= text_splitter.split_documents(data)
embeddings_api_model = OpenAIEmbeddings()

db_embed = FAISS.from_documents(texts, embeddings_api_model)

retriever= db_embed.as_retriever()

In [63]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    'search_web',
    'Searches and returns the data from the page',
)
tools = [tool]

In [66]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [67]:
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

agent = create_openai_tools_agent(llm=llm,tools=tools,prompt=prompt)
agent_executor = AgentExecutor(agent=agent,tools = tools)

In [68]:
res = agent_executor.invoke({'input' : "What is the name of the burger restauraunt from this webpage?"})
res['output']

'The burger restaurant mentioned on the webpage is "Smash Burgers" located in Belgrade.'

In [69]:
res1 = agent_executor.invoke({'input' : "What is the most expensive burger that is served today and what are its ingredients?"})
res1['output'] 

'The most expensive burger served today is the "Triple-Triple Western Smash burger" priced at RSD 1,270.00. The ingredients of this burger include 6 x 60g beef, emmental cheese, iceberg lettuce, onion, crispy bacon, and western sauce.'

In [70]:
res1 = agent_executor.invoke({'input' : "Can you make up the menu from the items provided on the page? it must include a burger, a drink, a dessert"})
res1['output']

'Here is a menu that includes a burger, a drink, and a dessert:\n\n- **Burger**: Double Classic Smash burger\n- **Drink**: Coca-Cola 0.33l\n- **Dessert**: Choco Brownie\n\nEnjoy your meal!'

In [71]:
res4 = agent_executor.invoke({'input' : "Can you make up the menu from the items provided on the page? it must include a burger, additional meal, a drink, a dessert. The total cost of all the meals should be below 1500 and I want you to write down the prices for all the items"})
res4['output']

'Here is a menu that includes a burger, additional meal, a drink, and a dessert with prices for each item:\n\n1. Burger:\n   - Double Classic Smash burger: RSD 760.00\n\n2. Additional Meal:\n   - Mozzarella Sticks (4 pieces) with sauce: RSD 450.00\n\n3. Drink:\n   - Coca-Cola 0.33l: RSD 250.00\n\n4. Dessert:\n   - Choco Brownie: RSD 490.00\n\nTotal Cost: RSD 1,950.00\n\nUnfortunately, the total cost of all the meals exceeds 1500. Would you like me to adjust the menu to meet the budget requirement?'

## 3. Advanced Prompt Techniques

### 3.1 Generated knowledge technique

prompt about X --> make 3 questions about X --> answer them from the X-expert standpoint --> make a response 

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
import os
from getpass import getpass

# set up envir variable
os.environ['OPENAI_API_KEY'] = getpass('OPENAI key here: ')

#### make the simple prompt

In [4]:
from langchain_openai import ChatOpenAI

prompt_text = "You're the evolutionary biologist and a professor in the university owning your own lab. Write the article about {topic}"

prompt = PromptTemplate.from_template(prompt_text)

llm = ChatOpenAI(temperature=0.5)
chain = prompt | llm | StrOutputParser()
res = chain.invoke('Evolutionry theory')
print(res)

Evolutionary theory is a fundamental concept in biology that explains how organisms change and adapt over time. Developed by Charles Darwin in the 19th century, this theory has revolutionized our understanding of the natural world and continues to be a cornerstone of modern biology.

At its core, evolutionary theory posits that all living organisms are descended from a common ancestor and have evolved through a process of natural selection. This process occurs when individuals with advantageous traits are more likely to survive and reproduce, passing those traits on to their offspring. Over time, these advantageous traits become more common in a population, leading to the gradual change and adaptation of species.

One of the key components of evolutionary theory is the concept of genetic variation. Within a population, there is a range of genetic diversity that arises from mutations, genetic recombination, and other processes. This variation provides the raw material for natural select

#### make advanced prompt . questions

In [5]:
# gather 3 questions from the answer into a list
def split_q(questions):
    return questions.split('\n')

prompt_text_1 = "You're the evolutionary biologist and a professor in the field. Ask three questions about the topic: {topic}. Split questions with line break"

topic = 'Evolutionary theory'
prompt_1 = PromptTemplate.from_template(prompt_text_1)

chain_1 = prompt_1 | llm | StrOutputParser() | split_q

questions = chain_1.invoke('Evolutionary biology')
print(questions)

['1. What are the key mechanisms of evolution and how do they drive changes in populations over time?', '2. How does natural selection play a role in shaping the genetic diversity of species and driving adaptation to changing environments?', '3. What are some current research topics in evolutionary biology that are pushing the boundaries of our understanding of the evolutionary process?']


#### put questions into a prompt as context

In [6]:
prompt_text_2 ='''As an evolutionary biologist and a professor in the field, your task is to write about the topic: {topic}.
Make clickbate heading for it. While providing the answer follow the template
Question: {question}
Answer: '''

prompt_2 = PromptTemplate.from_template(prompt_text_2)

chain_2 = prompt_2 | llm | StrOutputParser()

In [7]:
answers = chain_2.batch([{'topic' : topic, 'question' : questions[i]} for i in range(len(questions))])
print(answers)

["Uncover the Secrets of Evolutionary Theory: Key Mechanisms Explained!\n\nEvolutionary theory is based on the understanding that populations of organisms change over time due to several key mechanisms. These mechanisms include natural selection, genetic drift, gene flow, and mutation.\n\nNatural selection is perhaps the most well-known mechanism of evolution, where individuals with advantageous traits are more likely to survive and reproduce, passing on their genes to the next generation. This process leads to the gradual accumulation of beneficial traits in a population over time.\n\nGenetic drift, on the other hand, refers to random changes in gene frequencies within a population, often due to chance events. This can lead to the loss of genetic diversity and the fixation of certain alleles in a population.\n\nGene flow occurs when individuals from different populations interbreed, leading to the exchange of genetic material. This can introduce new genetic variation into a population

In [9]:
for i in answers:
    print(i)

Uncover the Secrets of Evolutionary Theory: Key Mechanisms Explained!

Evolutionary theory is based on the understanding that populations of organisms change over time due to several key mechanisms. These mechanisms include natural selection, genetic drift, gene flow, and mutation.

Natural selection is perhaps the most well-known mechanism of evolution, where individuals with advantageous traits are more likely to survive and reproduce, passing on their genes to the next generation. This process leads to the gradual accumulation of beneficial traits in a population over time.

Genetic drift, on the other hand, refers to random changes in gene frequencies within a population, often due to chance events. This can lead to the loss of genetic diversity and the fixation of certain alleles in a population.

Gene flow occurs when individuals from different populations interbreed, leading to the exchange of genetic material. This can introduce new genetic variation into a population and preve

#### put 3 (question, answer) pairs into the next prompt

In [11]:
context = '\n\n'.join([question+'\n'+answer for question, answer in zip(questions, answers)])
context

'1. What are the key mechanisms of evolution and how do they drive changes in populations over time?\nUncover the Secrets of Evolutionary Theory: Key Mechanisms Explained!\n\nEvolutionary theory is based on the understanding that populations of organisms change over time due to several key mechanisms. These mechanisms include natural selection, genetic drift, gene flow, and mutation.\n\nNatural selection is perhaps the most well-known mechanism of evolution, where individuals with advantageous traits are more likely to survive and reproduce, passing on their genes to the next generation. This process leads to the gradual accumulation of beneficial traits in a population over time.\n\nGenetic drift, on the other hand, refers to random changes in gene frequencies within a population, often due to chance events. This can lead to the loss of genetic diversity and the fixation of certain alleles in a population.\n\nGene flow occurs when individuals from different populations interbreed, lea

In [12]:
prompt_text_3 = '''You are the evolutionary biologist, write a short explanation (may it be less than 15 sentences) to the topic: {topic}
make your explanation based on three questions and three answers given in the context
Answers to the questions: {context}'''

prompt_3 = PromptTemplate.from_template(prompt_text_3)

chain_3 = prompt_3 | llm | StrOutputParser()

In [13]:
res_final =chain.invoke({'topic' : 'Evolutionary biology', 'context' : context})
print(res_final)

Evolutionary biology is a fascinating field of study that seeks to understand the processes by which organisms have evolved and diversified over time. As an evolutionary biologist and professor at a university with my own lab, I have dedicated my career to unraveling the mysteries of evolution and sharing my findings with the scientific community.

One of the key concepts in evolutionary biology is natural selection, which was famously proposed by Charles Darwin in his seminal work "On the Origin of Species." Natural selection is the process by which organisms that are better adapted to their environment are more likely to survive and reproduce, passing on their advantageous traits to future generations. This results in the gradual accumulation of beneficial traits over time, leading to the diversity of life that we see on Earth today.

Another important aspect of evolutionary biology is genetic drift, which refers to the random changes in the frequency of alleles (different versions o